In [3]:
import plotly.offline as plt
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Scatter
from ipywidgets import *
import ipywidgets as widgets

plt.init_notebook_mode(connected=True)

In [4]:
class Curves(object):
    """Class for showing scatter plot of intensities for one residue"""
    def __init__(self, titration):
        """Defines attributs
        argument 'titration' is a dicitionnary of 2 dimensions list for each key (should be a titration object from shiftome after developement)"""
        self.titration=titration
        self.residueSelector=widgets.Dropdown(options=[key for key in self.titration],
                                              description='Residue Number:',
                                              disabled=False,) # widget for selecting a specific residue
        self.intensity=widgets.FloatSlider(max=50, 
                                           step=1, 
                                           orientation="horizontal", 
                                           value=0) # slider widget for setting y ax max value
    def scatterPlot(self, residueNumber, maxValue):
        """Method that traces scatter plot of intensity at each Ligand/Protein ratio. 
        Creates a scatter plot.Sets the max value. Defines a legend.
        argument 'residueNumber' is a float number representing dictionnary key of titration.
        argument 'maxValue' is an integer representing the max value for y ax"""
        trace=go.Scatter(x=self.titration[residueNumber][0], y=self.titration[residueNumber][1],)
        data=[trace]
        ##sets default max value
        if maxValue ==0:
            intensityMax=1.2*max(self.titration[residueNumber][1])
        ##
        ##sets chosen max value
        else:
            intensityMax=maxValue
        ##
        fig = go.Figure(data=data, layout=dict(title='Titration Curve of residue '+str(residueNumber),annotations=[{
                                            'xref': 'paper', #plot number
                                            'yref': 'paper',
                                            'x': 0, #x position of x ax legend
                                            'xanchor': 'right',
                                            'y': 1,
                                            'yanchor': 'bottom',
                                            'text': 'Chemical shift intensity',
                                            'font':{'size':16},
                                            'showarrow': False
                                          }, {
                                            'xref': 'paper',
                                            'yref': 'paper',
                                            'x': 1,
                                            'xanchor': 'left',
                                            'y': 0,
                                            'yanchor': 'top',
                                            'text': 'Ligand/Protein ratio',
                                            'font':{'size':16},
                                            'showarrow': False
                                          }],  width=1000, #width of plotly interface
                                    height=500,
                                    margin=go.Margin(
                                        l=200,
                                        r=200,
                                        b=100,
                                        t=100,
                                        pad=4)),)
        fig['layout'].update(xaxis={'showline':True, 'showgrid':True,'zeroline':True}, 
                             yaxis={'range':[0,intensityMax],'showline':True,'zeroline':False}) #show x and y axes and sets y scale
        plt.iplot(fig)

c=Curves({1:[[1,2,3,4,5,6,7,8],[10,12,14,15,6,20,1,12,]], 2:[[1,2,3,4,5,6,7,8],[12,15,14,16,1,21,8,6,]], 3:[[1,2,3,4,5,6,7,8],[12,11,10,14,15,16,17,15]], '4':[[1,2,3,4,5,6,7,8],[11,14,16,20,25,23,24,48]]})
widgets.interactive(c.scatterPlot, residueNumber=c.residueSelector, maxValue=c.intensity)

A Jupyter Widget